In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time

In [2]:
def nothing(x):
    pass

cv2.namedWindow("Trackbars",cv2.WINDOW_NORMAL)

cv2.createTrackbar("L - H","Trackbars",0,179,nothing)
cv2.createTrackbar("L - S","Trackbars",0,255,nothing)
cv2.createTrackbar("L - V","Trackbars",0,255,nothing)
cv2.createTrackbar("U - H","Trackbars",179,179,nothing)
cv2.createTrackbar("U - S","Trackbars",255,255,nothing)
cv2.createTrackbar("U - V","Trackbars",255,255,nothing)

while True:
    frame = cv2.imread('shapescontour.jpg')
    
    l_h = cv2.getTrackbarPos("L - H","Trackbars")
    l_s = cv2.getTrackbarPos("L - S","Trackbars")
    l_v = cv2.getTrackbarPos("L - V","Trackbars")
    u_h = cv2.getTrackbarPos("U - H","Trackbars")
    u_s = cv2.getTrackbarPos("U - S","Trackbars")
    u_v = cv2.getTrackbarPos("U - V","Trackbars")
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    lower_range = np.array([l_h,l_s,l_v])
    upper_range = np.array([u_h,u_s,u_v])
    
    mask = cv2.inRange(hsv,lower_range,upper_range)
    
    res = cv2.bitwise_and(frame,frame,mask=mask)
    mask3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    stacked = np.hstack((mask3,frame,res))
    
    cv2.imshow("Trackbars",cv2.resize(stacked,(0,0),fx=0.4,fy=0.4))
    k = cv2.waitKey(1)
    if k == 27:
        break
    if k == ord('s'):
        thearray = [[l_h,l_s,l_v], [u_h,u_s,u_v]]
        print(thearray)
        
        np.save('color',thearray)
        break
        
cv2.destroyAllWindows()

[[112, 75, 75], [119, 255, 255]]


In [3]:
useload = True

if useload:
    colorval = np.load('color.npy')

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))


while True:
    frame = cv2.imread('shapescontour.jpg')
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    if useload:
        lower_range = colorval[0]
        uppen_range = colorval[1]
    else:
        lower_range = [15, 74, 125]
        uppen_range = [30, 255, 255]
        
    mask = cv2.inRange(hsv,lower_range,uppen_range)
    mask = cv2.erode(mask,kernel,iterations=1)
    mask = cv2.dilate(mask,kernel,iterations=2)
    
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours and cv2.contourArea(max(contours,key=cv2.contourArea)) > 1000:
        cnt = max(contours,key=cv2.contourArea)
        
        cv2.drawContours(frame, [cnt], 0, (0,255,0), 3) 

#         x2,y2,w,h = cv2.boundingRect(cnt)
#         cv2.rectangle(frame,(x2,y2),(x2+w,y2+h),(0,255,0),2)
    
    cv2.imshow('image',frame)
    
    k = cv2.waitKey(1)
    if k == 27:
        break
    

cv2.destroyAllWindows()